In [1]:
import logging

import probtorch
import torch

import combinators
import smc
import hmm
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
generative = smc.ParticleTrace(1)

In [3]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float).unsqueeze(0) * 2,
        'scale': torch.ones(1, 5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(1, 5),
        'scale': torch.ones(1, 5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(1, 5)}

In [4]:
init_hmm = combinators.Model(f=hmm.init_hmm, theta=hmm_params)
init_hmm.condition(generative)

In [5]:
z0, mu, sigma, pi, pi0 = init_hmm(T=50)

In [6]:
hmm_step = combinators.Model(f=hmm.hmm_step)
hmm_run = combinators.Model.sequence(hmm_step, 50, z0, mu, sigma, pi, pi0)

In [7]:
hmm_run.condition(generative)

In [8]:
z_last, mu, sigma, pi, pi0 = hmm_run()

In [9]:
generative = generative.squeeze()

In [10]:
num_particles = 100

In [11]:
smc_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    smc_hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [12]:
smc_hmm = smc.smc(hmm.hmm_step, hmm.hmm_retrace)

In [13]:
samples, elbos, inference = smc.particle_mh(num_particles, hmm.init_hmm, smc_hmm, 1000, 50, smc_hmm_params, generative, use_cuda=False)

In [14]:
for t in range(50):
    t += 1
    key = 'Z_%d' % t
    accuracy = (inference[key].value == generative[key].value).sum().to(dtype=torch.float) / inference.num_particles
    print('SMC accuracy at time %d: %f' % (t, accuracy))

SMC accuracy at time 1: 0.000000
SMC accuracy at time 2: 0.000000
SMC accuracy at time 3: 0.060000
SMC accuracy at time 4: 0.060000
SMC accuracy at time 5: 1.000000
SMC accuracy at time 6: 0.000000
SMC accuracy at time 7: 0.940000
SMC accuracy at time 8: 0.060000
SMC accuracy at time 9: 1.000000
SMC accuracy at time 10: 0.000000
SMC accuracy at time 11: 0.000000
SMC accuracy at time 12: 0.000000
SMC accuracy at time 13: 1.000000
SMC accuracy at time 14: 0.940000
SMC accuracy at time 15: 0.000000
SMC accuracy at time 16: 0.000000
SMC accuracy at time 17: 0.060000
SMC accuracy at time 18: 0.060000
SMC accuracy at time 19: 1.000000
SMC accuracy at time 20: 0.060000
SMC accuracy at time 21: 0.000000
SMC accuracy at time 22: 0.060000
SMC accuracy at time 23: 1.000000
SMC accuracy at time 24: 1.000000
SMC accuracy at time 25: 0.000000
SMC accuracy at time 26: 0.720000
SMC accuracy at time 27: 0.660000
SMC accuracy at time 28: 0.720000
SMC accuracy at time 29: 0.150000
SMC accuracy at time 30

In [15]:
elbos

tensor([-180.9284, -156.6157, -151.4902, -151.4902, -143.5333, -143.5333,
        -143.5333, -143.5333, -144.3280, -144.3280, -147.5277, -142.9639,
        -142.9639, -142.9639, -138.3296, -138.3296, -138.3296, -138.3296,
        -138.3296, -138.3296, -138.3296, -138.3296, -138.3296, -138.3296,
        -138.3296, -138.3296, -138.3296, -138.3296, -138.3296, -138.3296,
        -138.3296, -138.3296, -138.3296, -138.3296, -138.3296, -139.6300,
        -139.6300, -139.6300, -139.6300, -140.7809, -140.7809, -140.7809,
        -140.7809, -140.7809, -140.7809, -140.7809, -140.7809, -139.7697,
        -139.7697, -139.7697, -139.7697, -139.7697, -139.7697, -139.7697,
        -139.7697, -139.7697, -139.7697, -140.0739, -140.0739, -140.0739,
        -140.0739, -140.0739, -140.0739, -139.5882, -139.5882, -139.5882,
        -139.5882, -139.5882, -139.5882, -139.5882, -139.5882, -139.5882,
        -139.5882, -139.5882, -139.5882, -139.5882, -139.5882, -139.5882,
        -139.5882, -139.5882, -139.588